# Developing a RESTful API with FastAPI

```{admonition} Attribution
Notes on Chapter 3: Developing a RESTful API with FastAPI of {cite}`Voron2021`.
```

**Goals** 
   * Create working API endpoint and test it locally with HTTPie
   * Handling request parameters and customizing a response
   * Structuring a bigger project with routes

## Hello, world!

Let us quickly create a simple endpoint which has a GET method.

```python
# main.py
from fastapi import FastAPI

app = FastAPI()

@app.get("/")
async def hello_world():
    return {"hello": "world"}
```

The path function `hello_world` contains our route logic for the path `/` specified in the decorator. The decorator also specifies what HTTP method this function implements. The return value is automatically handled by FastAPI to produce a proper HTTP response with a JSON payload.

Here `app` is the main application object that will wire all of the API routes. We will 
start the server in the terminal as follows: 

```
$ uvicorn main:app
INFO:     Started server process [14121]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
```

Then, we perform the following GET request.

In [54]:
import requests
root = "http://127.0.0.1:8000"
path = "/"

response = requests.get(root + path)
response.json()

{'hello': 'world'}

## HTTPie

Before delving into the details of building REST APIs, we need to have a tool for making HTTP requests. We will be using **HTTPie**, a command-line tool aimed at making HTTP
requests with an intuitive syntax, JSON support, and syntax highlighting.

> HTTPie (pronounced *aitch*-*tee-tee-pie*) is a command-line HTTP client. Its goal is to make CLI interaction with web services as human-friendly as possible. HTTPie is designed for testing, debugging, and generally interacting with APIs & HTTP servers. The `http` & `https` commands allow for creating and sending arbitrary HTTP requests. They use simple and natural syntax and provide formatted and colorized output.

The general form of an HTTPie request is:

```
http [flags] [METHOD] URL [ITEM [ITEM]]
```

For our local server, we can call:

In [57]:
!http -v GET http://127.0.0.1:8000/

GET / HTTP/1.1
Accept: */*
Accept-Encoding: gzip, deflate, br
Connection: keep-alive
Host: 127.0.0.1:8000
User-Agent: HTTPie/2.6.0



HTTP/1.1 200 OK
content-length: 17
content-type: application/json
date: Thu, 23 Dec 2021 10:51:09 GMT
server: uvicorn

{
    "hello": "world"
}




The flag `-v` or `--verbose` is used here to print both the request and response. HTTPie also provides an `https` executable for dealing with URLs with `https://`. 

### Optional GET and POST

The `METHOD` argument is optional, and when you don’t specify it, HTTPie defaults to:

- `GET` for requests without body
- `POST` for requests with body

For example, the following is a GET request. 

In [ ]:
!http -v pie.dev/get 

On the other hand, the following request has data, so that the following defaults to a POST request:

In [ ]:
!http -v pie.dev/post hello=world

### Querystring parameters 

HTTPie provides `param==value` syntax for appending URL querystring parameters. With that, you don’t have to worry about escaping the `&` separators for your shell. The following are equivalent:

In [ ]:
!http https://api.github.com/search/repositories q==httpie per_page==1
!http "https://api.github.com/search/repositories?q=httpie&per_page=1"

### URL shortcuts for localhost

Shorthand for `localhost` is supported. For example, `:8000` would expand to `http://localhost:8000`. If the port is omitted, then port 80 is assumed.

In [71]:
!http :8000

HTTP/1.1 200 OK
content-length: 17
content-type: application/json
date: Thu, 23 Dec 2021 10:54:35 GMT
server: uvicorn

{
    "hello": "world"
}




## Automatic documentation

One of the most beloved features of FastAPI is the automatic interactive documentation.
If you open `http://localhost:8000/docs` in your browser, you should get a web
interface that looks similar to the following screenshot:

```{figure} ../../img/fastapi-docs.png
---
width: 40em
name: fastapi-docs
---

```

FastAPI automatically lists all defined endpoints and provide documentation about the expected inputs and outputs. You can even try each endpoint directly in this web interface.

## Handling request parameters

The main goal of a REST API is to provide a structured way 
   in which to interact with data. As such, it's crucial for the end user to send some 
   information to tailor the response they need, such as 

   - path parameters,
   - query parameters, 
   - body payloads, or 
   - headers.

To handle them, usually, web frameworks ask you to manipulate a request object to retrieve 
   the parts you are interested in and manually apply validation. However, that's not necessary 
   with FastAPI. Indeed, it allows you to define all of your parameters declaratively. Then, 
   it'll automatically retrieve them in the request and apply validations based on the type 
   hints as we will see below.

### Path parameters

We can have dynamic parameters in our paths which can then be passed to the path function. For example:

```python
# chapter3_path_parameters_01.py

@app.get("/users/{id}")
async def get_user(id: int):
    return {"id": id}
```

Then, we can make the following request for `id=123` (or for any other integer):

In [83]:
!http :8000/users/123

HTTP/1.1 200 OK
content-length: 10
content-type: application/json
date: Thu, 23 Dec 2021 14:35:30 GMT
server: uvicorn

{
    "id": 123
}




Notice the **type hint** in the path parameter `id`. If we pass a string into `id`, we get a response with a 422 status! Since this cannot be converted as a valid integer, the validation fails and outputs an error. All we need to do to trigger this validation is to type hint our parameter! Very cool.

#### Validation logic for path parameters

**Enumeration**. In the example below, `type` is a categorical parameter with two accepted values. We inherit from the `str` type and `Enum` class to facilitate the intended typing. We simply list the property name and its actual string value. 

```python
# chapter3_path_parameters_03.py

class UserType(str, Enum):
    STANDARD = "standard"
    ADMIN = "admin"


@app.get("/users/{type}/{id}")
async def get_user(id: int, type: UserType):
    return {"id": id, "type": type}
```


Note that the actual string value is what is passed in the `type` parameter (not the property name). If we pass a value that is not in the enumeration, we get an error.

In [119]:
!http :8000/users/admin/3

HTTP/1.1 200 OK
content-length: 23
content-type: application/json
date: Fri, 24 Dec 2021 07:15:25 GMT
server: uvicorn

{
    "id": 3,
    "type": "admin"
}




In [120]:
!http :8000/users/dog/3

HTTP/1.1 422 Unprocessable Entity
content-length: 184
content-type: application/json
date: Fri, 24 Dec 2021 07:15:47 GMT
server: uvicorn

{
    "detail": [
        {
            "ctx": {
                "enum_values": [
                    "standard",
                    "admin"
                ]
            },
            "loc": [
                "path",
                "type"
            ],
            "msg": "value is not a valid enumeration member; permitted: 'standard', 'admin'",
            "type": "type_error.enum"
        }
    ]
}




**Integer bounds**. For integers we can use the `Path` object from the `fastapi` library. In the example below, we set a lower bound to `id` so that it only takes positive values. 

```python
#chapter3_path_parameters_04.py
from fastapi import FastAPI, Path
app = FastAPI()

@app.get("/users/{id}")
async def get_user(id: int = Path(..., ge=1)):
    return {"id": id, "type": type}
```

i.e. `id >= 1`. Other possible arguments are `gt`, `lt`, `le`, etc. The `Path` function 
requires a first argument which becomes the default argument, using `...` indicates that 
we don't want to set a default argument. 


In [127]:
!http :8000/users/3

HTTP/1.1 200 OK
content-length: 8
content-type: application/json
date: Fri, 24 Dec 2021 07:25:49 GMT
server: uvicorn

{
    "id": 3
}




In [126]:
!http :8000/users/-1

HTTP/1.1 422 Unprocessable Entity
content-length: 149
content-type: application/json
date: Fri, 24 Dec 2021 07:25:35 GMT
server: uvicorn

{
    "detail": [
        {
            "ctx": {
                "limit_value": 1
            },
            "loc": [
                "path",
                "id"
            ],
            "msg": "ensure this value is greater than or equal to 1",
            "type": "value_error.number.not_ge"
        }
    ]
}




**Strings and regex**. We can bound string length using `min_length` and `max_length`. More generally, we can parse a string with a regular expressions in the `regex` argument.

```python
#chapter3_path_parameters_05.py

@app.get("/username/{username}")
async def get_username(username: str = Path(..., min_length=1, max_length=20)):
    return {"username": username}

@app.get("/license-plates/{license}")
async def get_license_plate(license: str = Path(..., regex=r"^\w{2}-\d{3}-\w{2}$")):
    return {"license": license}
```



Testing:

In [144]:
!http :8000/license-plates/AB-123-CD

HTTP/1.1 200 OK
content-length: 23
content-type: application/json
date: Fri, 24 Dec 2021 08:40:02 GMT
server: uvicorn

{
    "license": "AB-123-CD"
}




In [157]:
!http :8000/username/abcdefghijklmnopqrst # 20 characters

HTTP/1.1 200 OK
content-length: 35
content-type: application/json
date: Fri, 24 Dec 2021 08:41:16 GMT
server: uvicorn

{
    "username": "abcdefghijklmnopqrst"
}




```{admonition} Parameter metadata
Data validation is not the only option accepted by the parameter function `Path`. 
You can also set options such as `title`, `description`, and `deprecated`. These will add information about the parameter 
in the automatic documentation.

#### Query parameters

Query parameters are a common way to add some dynamic parameters to a 
URL. You find them at the end of the URL in the following form: `?param1=foo&param2=bar`. 
In a REST API, they are commonly used on read endpoints to apply pagination, a filter, a 
sorting order, or selecting fields.

 By default, arguments of path functions that are not path parameters are interpreted by 
    FastAPI as query parameters (i.e. without having to use the `Query` function defined 
    below).

```python
# chapter3_query_parameters_01.py

@app.get("/users/{type}/{id}")
async def user(
    type: UserType, 
    id: int = Path(..., ge=1), 
    page: int = 1,
    size: int = 1):
    return {
        "type": type,
        "id": id,
        "page": page, 
        "size": size,
    }
```